In [2]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from scipy import stats
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
import matplotlib.dates as mdates

def get_hstr(path):
    hstr = pd.read_csv(path)
    hstr['datetime'] = pd.to_datetime(hstr['CLCT_UNIX_TM'], unit='s', utc=True)
    hstr['datetime'] = hstr['datetime'].dt.tz_convert('Asia/Seoul').dt.strftime('%Y-%m-%d %H:%M:%S')
    hstr['datetime'] = hstr['datetime'].astype('datetime64[ns]')
    hstr.columns = ['현장교차로ID', '수집유닉스시각', '수집일시', '제어구분코드', '제어상태코드', '주기시간', '옵셋시간',
        'A링1현시시간', 'A링2현시시간', 'A링3현시시간', 'A링4현시시간', 'A링5현시시간', 'A링6현시시간',
        'A링7현시시간', 'A링8현시시간', 'B링1현시시간', 'B링2현시시간', 'B링3현시시간', 'B링4현시시간',
        'B링5현시시간', 'B링6현시시간', 'B링7현시시간', 'B링8현시시간', 'A링1현시보행시간',
        'A링2현시보행시간', 'A링3현시보행시간', 'A링4현시보행시간', 'A링5현시보행시간', 'A링6현시보행시간',
        'A링7현시보행시간', 'A링8현시보행시간', 'B링1현시보행시간', 'B링2현시보행시간', 'B링3현시보행시간',
        'B링4현시보행시간', 'B링5현시보행시간', 'B링6현시보행시간', 'B링7현시보행시간', 'B링8현시보행시간',
        '수집날짜시각']
    cols = list(hstr.columns).copy()
    cols.remove('수집날짜시각')
    cols.insert(1, '수집날짜시각')
    hstr = hstr[cols]
    return hstr

inter_nos = list(range(5031, 5048))

In [3]:
# 성남시 신호이력 데이터
s_tod_his = pd.read_csv('seongnam/S_TOD_HIS_1702597659892.csv')

# 인천시 신호이력 데이터
'''SELECT * FROM soitdspotintsoperhstr WHERE SPOT_INTS_ID BETWEEN 5031 AND 5047'''
csv_files = [f for f in os.listdir('incheon') if f.endswith('.csv')]

dfs = []
for file in csv_files:
    df_name = file.replace('.csv', '')
    print(df_name)
    globals()[df_name] = get_hstr(f'incheon/{file}')
    dfs.append(globals()[df_name])
    print(sorted([dt for dt in globals()[df_name]['수집날짜시각']])[0])
    print(sorted([dt for dt in globals()[df_name]['수집날짜시각']])[-1])
dfs = [df[(df.수집날짜시각.dt.hour!=23)&df.수집날짜시각.dt.minute!=59] for df in dfs]
hstr = pd.concat(dfs).drop_duplicates()
hstr = hstr[hstr.수집날짜시각.dt.day >= 12]
hstr = hstr[hstr.현장교차로ID.isin(inter_nos)].sort_values(by='수집날짜시각').reset_index(drop=True)
hstr = hstr[['현장교차로ID', '수집날짜시각', '제어구분코드', '제어상태코드', '주기시간', '옵셋시간',
             'A링1현시시간', 'A링2현시시간', 'A링3현시시간', 'A링4현시시간',
             'A링5현시시간', 'A링6현시시간', 'A링7현시시간', 'A링8현시시간',
             'B링1현시시간', 'B링2현시시간', 'B링3현시시간', 'B링4현시시간',
             'B링5현시시간', 'B링6현시시간', 'B링7현시시간', 'B링8현시시간']]
print(len(hstr))
print(sorted([dt for dt in hstr['수집날짜시각'] if dt.hour != 23])[0])
print(sorted([dt for dt in hstr['수집날짜시각'] if dt.hour != 23])[-1])
hstr = hstr.sort_values(by=['현장교차로ID', '수집날짜시각']).reset_index(drop=True)

soitdspotintsoperhstr_202312110921
2023-12-10 09:00:00
2023-12-11 23:59:59
soitdspotintsoperhstr_202312120814
2023-12-11 09:00:57
2023-12-12 23:59:55
soitdspotintsoperhstr_202312131102
2023-12-12 09:00:58
2023-12-13 23:59:56
soitdspotintsoperhstr_202312141254
2023-12-13 09:00:58
2023-12-14 23:59:59
soitdspotintsoperhstr_202312150926
2023-12-14 09:00:58
2023-12-15 23:59:30
soitdspotintsoperhstr_202312151700
2023-12-15 09:00:58
2023-12-15 23:59:30
32289
2023-12-12 00:00:09
2023-12-15 16:59:54


In [20]:
print(hstr.제어구분코드.unique())
print(hstr.제어상태코드.unique())
print(hstr.제어구분코드.value_counts())
print(hstr.제어상태코드.value_counts())
S0 = list(hstr[hstr.제어구분코드==0].index)
S3 = list(hstr[hstr.제어구분코드==3].index)
T0 = list(hstr[hstr.제어상태코드==0].index)
T1 = list(hstr[hstr.제어상태코드==1].index)
T2 = list(hstr[hstr.제어상태코드==2].index)
T3 = list(hstr[hstr.제어상태코드==3].index)

[3 0]
[1 2 0 3]
제어구분코드
3    26696
0     5593
Name: count, dtype: int64
제어상태코드
1    25944
0     5519
2      751
3       75
Name: count, dtype: int64


In [ ]:
# 제어구분코드 (0: 비감응 지역제어, 1: 감응 지역제어, 2: 감응 중앙제어, 3: 비감응 중앙제어)
# 제어상태코드 (0:오프라인, 1:온라인, 2:트랜스, 3:통신장애, 4:수동)
print(hstr.제어구분코드.value_counts())
print(hstr.제어상태코드.value_counts())
k=7
for j in range(2400,2500):
    print(j)
    i = S0[j]
    context = range(i - k + 1, i + k)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(hstr.iloc[context])

In [25]:
# k=7
# for j in range(10):
#     print(j)
#     i = T2[j]
#     context = range(i - k + 1, i + k)
#     with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#         display(hstr.iloc[context])

In [99]:
# 제어구분코드 (0: 비감응 지역제어, 1: 감응 지역제어, 2: 감응 중앙제어, 3: 비감응 중앙제어)
# 제어상태코드 (0:오프라인, 1:온라인, 2:트랜스, 3:통신장애, 4:수동)
k = 7

print(f"j : 0 ~ {len(S0)}")
print()

j = 0
print("j=0 : ..., 1, 0, 2, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 1
print("j=1 : ..., 1, 0, 2, 2, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 2
print("j=2 : ..., 1, 0, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 8
print("j=8 : ..., 1, 0, 0, 2, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 9
print("j=9 : ..., 1, 0, 0, 2, 2, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 11
print("j=11 : ..., 1, 0, 0, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 20
print("j=20 : ..., 1, 0, 0, 0, 2, 2, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 24
print("j=24 : ..., 1, 0, 0, 2, 2, 2, 2, 2, 3, 3, ...")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k + 90)])

j = 109
print("j=109 : ..., 1, 0, 0, 2, 2, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 111
print("j=111 : ..., 1, 0, 0, 2, 2, 2, 2, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k)])

j = 132
print("j=132 : ..., 1, 0, 0, ..., 0, 1, ...")
display(hstr.iloc[range(S0[j] - k + 1, S0[j] + k+1891)])

j : 0 ~ 5593

j=0 : ..., 1, 0, 2, 1, ...


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,...,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
773,5031,2023-12-13 10:23:40,3,1,160,140,49,20,27,29,...,0,0,49,20,27,29,35,0,0,0
774,5031,2023-12-13 10:26:20,3,1,160,140,49,20,27,29,...,0,0,49,20,27,29,35,0,0,0
775,5031,2023-12-13 10:29:00,3,1,160,140,49,20,27,29,...,0,0,49,20,27,29,35,0,0,0
776,5031,2023-12-13 10:31:40,3,1,160,140,49,20,27,29,...,0,0,49,20,27,29,35,0,0,0
777,5031,2023-12-13 10:34:20,3,1,160,140,49,20,27,29,...,0,0,49,20,27,29,35,0,0,0
778,5031,2023-12-13 10:39:07,3,1,31,107,49,20,27,156,...,0,0,49,20,27,156,35,0,0,0
779,5031,2023-12-13 10:41:27,0,0,140,87,37,16,27,25,...,0,0,37,16,27,25,35,0,0,0
780,5031,2023-12-13 10:45:00,3,2,213,140,69,31,31,43,...,0,0,69,31,31,43,39,0,0,0
781,5031,2023-12-13 10:47:40,3,1,160,140,49,20,27,29,...,0,0,49,20,27,29,35,0,0,0
782,5031,2023-12-13 10:50:20,3,1,160,140,49,20,27,29,...,0,0,49,20,27,29,35,0,0,0


j=1 : ..., 1, 0, 2, 2, 1, ...


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,...,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
907,5031,2023-12-13 17:53:38,3,1,180,159,65,20,27,33,...,0,0,65,20,27,33,35,0,0,0
908,5031,2023-12-13 17:56:38,3,1,180,159,65,20,27,33,...,0,0,65,20,27,33,35,0,0,0
909,5031,2023-12-13 17:59:38,3,1,180,159,65,20,27,33,...,0,0,65,20,27,33,35,0,0,0
910,5031,2023-12-13 18:02:38,3,1,180,159,65,20,27,33,...,0,0,65,20,27,33,35,0,0,0
911,5031,2023-12-13 18:05:38,3,1,180,159,65,20,27,33,...,0,0,65,20,27,33,35,0,0,0
912,5031,2023-12-13 18:10:47,3,1,52,107,65,20,27,161,...,0,0,65,20,27,161,35,0,0,0
913,5031,2023-12-13 18:13:07,0,0,140,67,37,16,27,25,...,0,0,37,16,27,25,35,0,0,0
914,5031,2023-12-13 18:17:06,3,2,239,126,90,31,31,48,...,0,0,90,31,31,48,39,0,0,0
915,5031,2023-12-13 18:20:39,3,2,213,159,78,25,31,40,...,0,0,78,25,31,40,39,0,0,0
916,5031,2023-12-13 18:23:39,3,1,180,160,65,20,27,33,...,0,0,65,20,27,33,35,0,0,0


j=2 : ..., 1, 0, 1, ...


...

,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,...,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
6831,5034,2023-12-13 21:28:57,3,1,170,157,51,30,45,44,...,0,0,29,52,45,44,0,0,0,0
6832,5034,2023-12-13 21:31:47,3,1,170,157,51,30,45,44,...,0,0,29,52,45,44,0,0,0,0
6833,5034,2023-12-13 21:34:37,3,1,170,157,51,30,45,44,...,0,0,29,52,45,44,0,0,0,0
6834,5034,2023-12-13 21:37:27,3,1,170,157,51,30,45,44,...,0,0,29,52,45,44,0,0,0,0
6835,5034,2023-12-13 21:40:17,3,1,170,157,51,30,45,44,...,0,0,29,52,45,44,0,0,0,0
6836,5034,2023-12-13 21:43:07,3,1,124,157,51,30,43,0,...,0,0,29,52,43,0,0,0,0,0
6837,5034,2023-12-13 21:43:51,0,0,90,32,0,0,0,90,...,0,0,0,0,0,90,0,0,0,0
6838,5034,2023-12-13 21:46:21,0,0,150,11,43,21,43,43,...,0,0,21,43,43,43,0,0,0,0
6839,5034,2023-12-13 21:48:51,3,2,150,161,43,21,43,43,...,0,0,21,43,43,43,0,0,0,0
6840,5034,2023-12-13 21:51:37,3,2,166,157,50,29,44,43,...,0,0,28,51,44,43,0,0,0,0


j=111 : ..., 1, 0, 0, 2, 2, 2, 2, 1


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,...,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
6860,5034,2023-12-13 22:45:16,3,1,159,156,47,26,43,43,...,0,0,27,46,43,43,0,0,0,0
6861,5034,2023-12-13 22:47:55,3,1,160,156,47,26,44,43,...,0,0,27,46,44,43,0,0,0,0
6862,5034,2023-12-13 22:50:35,3,1,160,156,47,26,44,43,...,0,0,27,46,44,43,0,0,0,0
6863,5034,2023-12-13 22:53:16,3,1,160,156,47,26,44,43,...,0,0,27,46,44,43,0,0,0,0
6864,5034,2023-12-13 22:55:56,3,1,160,156,47,26,44,43,...,0,0,27,46,44,43,0,0,0,0
6865,5034,2023-12-13 22:58:36,3,1,160,156,47,26,44,43,...,0,0,27,46,44,43,0,0,0,0
6866,5034,2023-12-13 23:02:06,0,0,109,46,0,0,55,54,...,0,0,0,57,55,54,0,0,0,0
6867,5034,2023-12-13 23:04:35,0,0,150,36,43,21,43,43,...,0,0,21,43,43,43,0,0,0,0
6868,5034,2023-12-13 23:07:06,3,2,150,26,43,21,43,43,...,0,0,21,43,43,43,0,0,0,0
6869,5034,2023-12-13 23:09:36,3,2,150,16,43,21,43,43,...,0,0,21,43,43,43,0,0,0,0


j=132 : ..., 1, 0, 0, ....


,현장교차로ID,수집날짜시각,제어구분코드,제어상태코드,주기시간,옵셋시간,A링1현시시간,A링2현시시간,A링3현시시간,A링4현시시간,...,A링7현시시간,A링8현시시간,B링1현시시간,B링2현시시간,B링3현시시간,B링4현시시간,B링5현시시간,B링6현시시간,B링7현시시간,B링8현시시간
21272,5041,2023-12-15 16:50:23,3,1,170,104,46,103,11,10,...,0,0,46,103,11,10,0,0,0,0
21273,5041,2023-12-15 16:53:13,3,1,170,104,46,103,11,10,...,0,0,46,103,11,10,0,0,0,0
21274,5041,2023-12-15 16:56:03,3,1,170,104,46,103,11,10,...,0,0,46,103,11,10,0,0,0,0
21275,5041,2023-12-15 16:58:53,3,1,170,104,46,103,11,10,...,0,0,46,103,11,10,0,0,0,0
21276,5042,2023-12-12 00:01:23,0,0,150,84,33,51,29,16,...,0,0,33,51,29,16,13,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23171,5042,2023-12-15 16:56:01,0,0,170,101,35,65,29,20,...,0,0,35,65,29,20,13,8,0,0
23172,5042,2023-12-15 16:58:51,0,0,170,101,35,65,29,20,...,0,0,35,65,29,20,13,8,0,0
23173,5043,2023-12-12 00:01:11,3,1,150,71,39,52,8,11,...,0,0,39,52,8,11,40,0,0,0
23174,5043,2023-12-12 00:03:41,3,1,150,71,39,52,8,11,...,0,0,39,52,8,11,40,0,0,0
